In [2]:
import numpy as np
import sklearn 
import pandas as pd
import matplotlib.pyplot as plt
import scipy
from scipy.integrate import odeint
import math
from statsmodels.tsa.arima.model import ARIMA

#import csvs
train = pd.read_csv("../../data/train.csv")
test = pd.read_csv("../../data/test.csv")
graph = pd.read_csv("../../data/graph.csv")

statesdata = {}
states = pd.Series.unique(train['Province_State'])
num_states = len(states)
for s in states:
    statesdata[s] = train.loc[train['Province_State'] == s ,:]

In [3]:
#p,d,q = 5,2,3
days = 26

def predictARIMA(X, p, d, q, days):
    model = ARIMA(X, order=(p,d,q))
    model_fit = model.fit()
    start = len(X)
    forecast = model_fit.predict(start = start, end = start + days)
    return(forecast)

# def difference(dataset, interval = 1):
#     diff = list()
#     for i in range(interval, len(dataset)):
#         value = dataset[i]-dataset[i-interval]
#         diff.append(value)
#     return np.array(diff)

# # invert differenced value
# def inverse_difference(history, yhat, interval=1):
#     #yhat = np.exp(yhat)
#     return yhat + history[-1]


proj = {}
for s in states:
    a = statesdata[s]
    a = a.reset_index()
    confirmed = a['Confirmed']
    deaths = a['Deaths']
    
    X = confirmed.values
    
    forecastC = predictARIMA(X, 2,2,1, days)
    # BEST PARAMS SO FAR ARE 2,2,1
    
    Y= deaths.values


    forecastD = predictARIMA(Y, 4,2,3, days)
    # BEST PARAMS SO FAR ARE 4,2,3

    df = {'Confirmed': forecastC, 'Deaths': forecastD}
    
    proj[s] = pd.DataFrame(df)

/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model

In [4]:
proj

{'Alabama':         Confirmed       Deaths
 0   127308.837196  2196.399461
 1   128513.398335  2209.477139
 2   129751.125389  2234.926038
 3   130984.830270  2257.315839
 4   132221.112886  2285.938123
 5   133457.050615  2308.779484
 6   134693.189191  2331.507829
 7   135929.298506  2351.241565
 8   137165.423507  2371.507484
 9   138401.546046  2392.643919
 10  139637.669813  2414.940166
 11  140873.793375  2438.068778
 12  142109.917032  2460.950315
 13  143346.040673  2483.421347
 14  144582.164321  2505.215862
 15  145818.287968  2526.811781
 16  147054.411615  2548.459585
 17  148290.535262  2570.413210
 18  149526.658909  2592.621931
 19  150762.782556  2614.929336
 20  151998.906204  2637.172147
 21  153235.029851  2659.266063
 22  154471.153498  2681.246364
 23  155707.277145  2703.190843
 24  156943.400793  2725.180376
 25  158179.524440  2747.240380
 26  159415.648087  2769.352282,
 'Alaska':       Confirmed     Deaths
 0   5320.022881  37.763348
 1   5387.169416  38.31339

In [22]:
order = test.loc[0:49,'Province_State']

# format submission
conf = []
dead = []
fid = 0
for i in range(days):
    for j in order:
        projection = proj[j].iloc[i]
        #print(j, 'day', i)
        conf.append(int(projection['Confirmed']))
        dead.append(int(projection['Deaths']))
        #print(fid)
        fid+=1

test['Confirmed'] = conf
test['Deaths'] = dead

In [23]:
submission = test.drop(columns=['Province_State', 'Date'])
print(submission)

ForecastID  Confirmed  Deaths
0              0     127308    2196
1              1       5320      37
2              2     202260    5058
3              3      61767     811
4              4     717990   13150
...          ...        ...     ...
1295        1295     145806    2951
1296        1296      87103    2101
1297        1297      13603     314
1298        1298      91833    1256
1299        1299       4747      43

[1300 rows x 3 columns]


In [24]:
submission.to_csv('team25.csv', index = False, header = True)
